In [1]:
"""
slideshare のファイルを pdf 抽出ダウンロード

requirements
# pip install img2pdf
# pip install beautifulsoup
"""
pass

In [2]:
import os
import img2pdf
from os import listdir, walk
from os.path import isfile, join
import shutil
import urllib.request
from bs4 import BeautifulSoup 

def download_images(url, out_path, dir_work="/tmp"):
    # 作業ディレクトリを一旦クリア
    if os.path.exists(dir_work):
        shutil.rmtree(dir_work)
    os.makedirs(dir_work)
    
    html = urllib.request.urlopen(url)
    soup = BeautifulSoup(html, "html.parser")
    title = 'pdf_images' #soup.title.string
    images = soup.findAll('img', {'class':'slide_image'})
    print("ダウンロード中:", url)
    filenames = []
    for image in images:
        image_url = image.get('data-full').split('?')[0]
        command = 'wget %s -P %s' % (image_url, join(dir_work, title))
        #print("command:", command)
        os.system(command)
        
        filenames.append(os.path.basename(image_url))
    
    print("ダウンロード完了")

    convert_pdf(out_path, join(dir_work, title), filenames)
    # 作業ディレクトリは削除
    shutil.rmtree(join(dir_work, title))

def convert_pdf(out_path, dir_path, filenames):
    print("pdf コンバート中")
    f = ["%s/%s" % (dir_path, x) for x in filenames]

    pdf_bytes = img2pdf.convert(f, dpi=300, x=None, y=None)
    doc = open(out_path, 'wb')
    doc.write(pdf_bytes)
    doc.close()
    print("pdf コンバート完了")


In [3]:
# 作業用のディレクトリ　　一旦ファイルをダウンロードする先
DIR_WORK = "/exdisk/workspace/20170729-04"
# ダウンロードしたファイルを結合して pdf を出力するパス
OUT_PATH = "/var/www/html/notedoc/result.pdf"
# 出力先を html でアクセスする際の URL
URL_LINK = "http://spark01.nel-t.net/notedoc/result.pdf"

#  抽出対象の slideshare のURL
url = input('Slideshare URL : ')
download_images(url, OUT_PATH, dir_work=DIR_WORK)

Slideshare URL : https://www.slideshare.net/ssusere55c63/variational-autoencoder-64515581?qid=e8e30874-e0a4-42d4-9422-cfd574f5730e&v=&b=&from_search=1
ダウンロード中: https://www.slideshare.net/ssusere55c63/variational-autoencoder-64515581?qid=e8e30874-e0a4-42d4-9422-cfd574f5730e&v=&b=&from_search=1
ダウンロード完了
pdf コンバート中
pdf コンバート完了


In [4]:
from IPython.core.display import display, HTML
display(HTML('<a href="' + URL_LINK + '" target=new>extracted pdf </a>'))